In [186]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def cstruct(Q,Qs,Qr,send,results):
    s_thread=send['Thread']
    s_index=send['Index']
    if s_index==1:
        return results
    temp=Q[(Q.Thread == s_thread) & (Q.Index==s_index-1)].iloc[0]
    if temp['Event']=='send':
        results.append(temp['ID'])
        send=temp
        return cstruct(Q,Qs,Qr,send,results)
    else:
        results.append(temp['ID'])
        send=Qs.loc[temp.name]
        results.append(send['ID'])
        return cstruct(Q,Qs,Qr,send,results)
    
def no_index(start,finish,t,k):
    for l in range(start,finish):
        if t[l]>0 and R[l] in cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[k])].iloc[0],results):
            return True
        else:
            continue
    return False

'''
if len(t[t>0])>0 and cstruct(Q,Qs,Qr,Qs.iloc[k],results):
    return False #there is an index
else:
    return True # there is no index
'''
    
#============ここから定義============
    
heading_res=('Thread','Port','Index')
heading_snd=('Thread','Port','Event','Index')


#Q is SYN-sequence
Qr=pd.read_table('SYN-res.csv',names=heading_res)
Qs=pd.read_table('SYN-snd.csv',names=heading_snd)


r_list=list(range(1,len(Qr)+1))
r_list=list(map(lambda x:'r'+str(x),r_list)) #['r1', 'r2', 'r3', 'r4']
s_list=list(range(1,len(Qs)+1))
s_list=list(map(lambda x:'s'+str(x),s_list)) #['s1', 's2', 's3', 's4']

Qr.insert(0,'ID',r_list) #attach the name of event
Qs.insert(0,'ID',s_list) #attach the name of event

Q=[] #Q is SYN-sequence
Q=pd.merge(Qr,Qs,how='outer')
race_set={'r1':['s2'],'r2':[],'r3':['s4'],'r4':[]} #sample data



#============ここまで定義============

R=[] #create race_set
D=[] #number of race_set
heading=[] #{r1,r2,....r}

# set R,D
for key in race_set.keys():
    if len(race_set[key])>0:
        R.append(key) # raceのあるイベントを追加
        D.append(len(race_set[key])) #raceの数を追加
        heading.append(key) # r
table=pd.DataFrame([],columns=heading)

t=np.zeros(len(R)) #raceの数分の配列

while True:
    results=[]
    #num=0
    #num+=1　
    max_index=''
    for i in range(len(t)-1,-1,-1):
        if t[i]<D[i] and t[i]!=-1:
            max_index=i
    if max_index=='':
        break
    t[i]+=1
    
    if t[i]==1: #just changed t[i] from 0 to 1
        
        for j in tqdm(range(i+1,len(R))):
            if t[j]!=-1 and cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[j])].iloc[0],results):
                t[j]=-1
    for j in tqdm(range(i+1,len(R))):
        if(t[j]==D[j]):
            t[j]=0 #just change t[j] from dj to 0
            
            for k in tqdm(range(j+1,len(R))):
                if t[k]==-1 and R[j] in cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[k])].iloc[0],results) and no_index(1,k,t,Qr[(Qr.ID==R[k])].iloc[0]):# ここ間違えてる可能性あり
                    t[k]=0
    print(str(i)+"\n")
    print(int(t[i]))
    #let s be the t[i] sending event in race_set(ri)
    s= race_set[R[i]][int(t[i])] 

    
    if no_index2(1,len(R)+1,t,s):
        table=table.append(pd.Series(t,index=table.columns),ignore_index=True)
table




100%|██████████| 1/1 [00:00<00:00, 1024.00it/s]

0

1


IndexError: list index out of range

In [12]:
# 例題

send=Qs.loc[2]
results=list(cstruct(Q[0],Qs,Qr,send,results))
print(results)

['r2', 's2']


In [97]:
for i in range(len(t)-1,-1,-1):
    print(i)

1
0


In [156]:
cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[0])].iloc[0],results)

[]

In [163]:
results=[]
cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[0])].iloc[0],results)

[]

In [157]:
Qr[(Qr.ID==R[0])].iloc[0]

ID          r1
Thread      T1
Port      {p2}
Index        1
Name: 0, dtype: object

In [192]:
cstruct(Q,Qs,Qr,Qr[(Qr.ID==R[0])].iloc[0],results)

[]

In [191]:
cstruct(Q,Qs,Qr,Qr.iloc[0],results)

[]

In [190]:
Qr.iloc[0]

ID          r1
Thread      T1
Port      {p2}
Index        1
Name: 0, dtype: object

In [193]:
Qr[(Qr.ID==R[0])].iloc[0]

ID          r1
Thread      T1
Port      {p2}
Index        1
Name: 0, dtype: object